## Deliverable 2. Create a Customer Travel Destinations Map.

In [95]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import google_api_key

# Configure gmaps API key
gmaps.configure(google_api_key)

In [77]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv", keep_default_na=False)
city_data_df.drop(columns='Unnamed: 0', inplace=True)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Grindavik,IS,63.8424,-22.4338,17.60,53,75,14.97,broken clouds
1,Jamestown,SH,-15.9387,-5.7168,71.42,76,3,17.00,clear sky
2,Khorixas,NA,-20.3667,14.9667,80.17,37,58,7.87,broken clouds
3,Cayenne,GF,4.9333,-52.3333,82.40,83,90,9.22,overcast clouds
4,Khatanga,RU,71.9667,102.5000,-11.61,86,10,9.01,clear sky


In [99]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
print("Hey there! I see you're looking for holiday destinations. Start your search by answering a few questions..")
min_temp = float(input('How much cold (in Fahrenheit) can you handle?'))
max_temp = float(input('How much heat (in Fahrenheit) can you handle?'))

Hey there! I see you're looking for holiday destinations. Start your search by answering a few questions..
How much cold (in Fahrenheit) can you handle?75
How much heat (in Fahrenheit) can you handle?85


In [100]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vaca_cities_df = city_data_df.loc[
                                (city_data_df['Max Temp'] >= min_temp) &
                                (city_data_df['Max Temp'] <= max_temp)
                                ].reset_index().drop(columns='index')
vaca_cities_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Khorixas,NA,-20.3667,14.9667,80.17,37,58,7.87,broken clouds
1,Cayenne,GF,4.9333,-52.3333,82.40,83,90,9.22,overcast clouds
2,Itaituba,BR,-4.2761,-55.9836,81.59,80,100,2.73,overcast clouds
3,Hilo,US,19.7297,-155.0900,75.20,64,90,8.05,moderate rain
4,Rikitea,PF,-23.1203,-134.9692,77.52,68,79,17.49,broken clouds


In [128]:
# 4a. Determine if there are any empty rows.
vaca_cities_df.count()

City                   207
Country                207
Lat                    207
Lng                    207
Max Temp               207
Humidity               207
Cloudiness             207
Wind Speed             207
Current Description    207
dtype: int64

In [85]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No empty rows found in df so skipping this step

In [130]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vaca_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.NaN
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Khorixas,NA,80.17,broken clouds,-20.3667,14.9667,NaN
1,Cayenne,GF,82.40,overcast clouds,4.9333,-52.3333,NaN
2,Itaituba,BR,81.59,overcast clouds,-4.2761,-55.9836,NaN
3,Hilo,US,75.20,moderate rain,19.7297,-155.0900,NaN
4,Rikitea,PF,77.52,broken clouds,-23.1203,-134.9692,NaN
...,...,...,...,...,...,...,...
202,Pucallpa,PE,84.20,broken clouds,-8.3791,-74.5539,NaN
203,Saint-Denis,RE,78.80,scattered clouds,-20.8667,55.4667,NaN
204,Ranong,TH,77.43,broken clouds,9.9658,98.6348,NaN
205,Samarai,PG,82.47,scattered clouds,-10.6167,150.6667,NaN


In [131]:
# Checking null value in Hotel Name column
hotel_df.count()

City                   207
Country                207
Max Temp               207
Current Description    207
Lat                    207
Lng                    207
Hotel Name               0
dtype: int64

In [132]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key
}

# 6b. Iterate through the hotel DataFrame.
for i in range(0,len(hotel_df)):
    # 6c. Get latitude and longitude from DataFrame
    params['location'] = f"{hotel_df['Lat'][i]},{hotel_df['Lng'][i]}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(base_url,params)
    results = response.json()['results']
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df['Hotel Name'][i] = results[0]['name']
        print(f"{i+1} | Found a hotel near {hotel_df['City'][i]}!")
    except:
        print(f"{i+1} | {hotel_df['City'][i]} does not have a hotel within 5KMs")
hotel_df

<ipython-input-132-64791d4e4095>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'][i] = results[0]['name']
/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


1 | Found a hotel near Khorixas!
2 | Found a hotel near Cayenne!
3 | Found a hotel near Itaituba!
4 | Found a hotel near Hilo!
5 | Found a hotel near Rikitea!
6 | Found a hotel near Cairns!
7 | Found a hotel near Buala!
8 | Found a hotel near Georgetown!
9 | Found a hotel near Lázaro Cárdenas!
10 | Found a hotel near Bambous Virieux!
11 | Found a hotel near Codajás!
12 | Found a hotel near Mahébourg!
13 | Found a hotel near Bubaque!
14 | Found a hotel near Carnarvon!
15 | Found a hotel near Port Blair!
16 | Found a hotel near Quatre Cocos!
17 | Found a hotel near Vaini!
18 | Found a hotel near Sibolga!
19 | Found a hotel near Vila Velha!
20 | Found a hotel near Atuona!
21 | Found a hotel near Loreto!
22 | Warmbad does not have a hotel within 5KMs
23 | Found a hotel near Souillac!
24 | Found a hotel near Faanui!
25 | Found a hotel near Chilca!
26 | Found a hotel near São João da Barra!
27 | Found a hotel near Tra Vinh!
28 | Found a hotel near Kapaa!
29 | Found a hotel near Butaritari!
3

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Khorixas,NA,80.17,broken clouds,-20.3667,14.9667,Khorixas Lodge
1,Cayenne,GF,82.40,overcast clouds,4.9333,-52.3333,Hôtel Le Dronmi
2,Itaituba,BR,81.59,overcast clouds,-4.2761,-55.9836,Hotel Campos
3,Hilo,US,75.20,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Rikitea,PF,77.52,broken clouds,-23.1203,-134.9692,Pension Maro'i
...,...,...,...,...,...,...,...
202,Pucallpa,PE,84.20,broken clouds,-8.3791,-74.5539,Hospedaje Komby
203,Saint-Denis,RE,78.80,scattered clouds,-20.8667,55.4667,Central Hôtel
204,Ranong,TH,77.43,broken clouds,9.9658,98.6348,Tinidee Hotel@Ranong
205,Samarai,PG,82.47,scattered clouds,-10.6167,150.6667,Nuli Sapi


In [133]:
hotel_df.count()

City                   207
Country                207
Max Temp               207
Current Description    207
Lat                    207
Lng                    207
Hotel Name             194
dtype: int64

In [137]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Khorixas,NA,80.17,broken clouds,-20.3667,14.9667,Khorixas Lodge
1,Cayenne,GF,82.40,overcast clouds,4.9333,-52.3333,Hôtel Le Dronmi
2,Itaituba,BR,81.59,overcast clouds,-4.2761,-55.9836,Hotel Campos
3,Hilo,US,75.20,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Rikitea,PF,77.52,broken clouds,-23.1203,-134.9692,Pension Maro'i
...,...,...,...,...,...,...,...
202,Pucallpa,PE,84.20,broken clouds,-8.3791,-74.5539,Hospedaje Komby
203,Saint-Denis,RE,78.80,scattered clouds,-20.8667,55.4667,Central Hôtel
204,Ranong,TH,77.43,broken clouds,9.9658,98.6348,Tinidee Hotel@Ranong
205,Samarai,PG,82.47,scattered clouds,-10.6167,150.6667,Nuli Sapi


In [138]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [147]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [148]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)
# 11b. Display the figure
fig = gmaps.figure(
    center = (30.0, 31.0),
    zoom_level = 1.75
    )
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))